# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd# импорт библиотеки pandas
data=pd.read_csv('/datasets/data.csv')# чтение файла с данными и сохранение в data
data.head(10)# получение первых 10 строк таблицы data
from pymystem3 import Mystem# подключим библиотеку с функцией лемматизации на русском языке
from collections import Counter# подсчёт числа упоминаний слов в тексте
m = Mystem() 

In [2]:
data.info()# получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Описание данных**

В таблице двенадцать столбцов

* `userID` — идентификатор пользователя;
* `children` - количество детей в семье;
* `days_employed` - общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита;

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы — данные о клиенте. Большая часть колонок описывает самого клиента. Остальные данные рассказывают о цели получения кредита. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
display(data.isna().sum()) # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
display(data.isna().mean()) # подсчёт пропусков

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [5]:
#Методом isna() найдём все строки с пропусками в столбце 'days_employed', 'total_income'и просмотрим первые пять.
display(data[data['days_employed'].isna()].head())  
display(len (data[data['days_employed'].isna()]))#сосчитаем строки без трудового стажа
display(data[data['total_income'].isna()].head()) 
display(len (data[data['total_income'].isna()]))#сосчитаем строки без ежемесячного дохода


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


2174

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


2174

In [6]:
display(data['income_type'].unique())# проверка типа занятости
display(data['income_type'].value_counts())# подсчет количества по типу занятости

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [7]:
# разбиваем пропуски по типу занятости
pensioner_income_type=data[data['income_type'] == 'пенсионер']
display (pensioner_income_type)
pensioner_total_income_median=data[data['income_type'] == 'пенсионер']['total_income'].median()
pensioner_days_employed_median=data[data['income_type'] == 'пенсионер']['days_employed'].median()
display(pensioner_total_income_median)
display(pensioner_days_employed_median)
# заменяем пропуски по типу занятости на медианные значения

data['days_employed']=data[data['income_type'] == 'пенсионер']['days_employed'].fillna(value=pensioner_days_employed_median)
data['total_income']=data[data['income_type'] == 'пенсионер']['total_income'].fillna(value=pensioner_total_income_median)
display (data[data['income_type'] == 'пенсионер'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


118514.48641164352

365213.3062657312

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,365213.306266,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [8]:
# разбиваем пропуски по типу занятости
employee_income_type=data[data['income_type'] == 'сотрудник']
display (employee_income_type)
display(data[data['income_type'] == 'сотрудник'].isna().sum())
employee_total_income_mean=data[data['income_type'] == 'сотрудник']['total_income'].mean()
employee_days_employed_mean=data[data['income_type'] == 'сотрудник']['days_employed'].mean()
display(employee_total_income_mean)
display(employee_days_employed_mean)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,NaN,42,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья
1,1,NaN,36,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
2,0,NaN,33,Среднее,1,женат / замужем,0,M,сотрудник,0,NaN,покупка жилья
3,3,NaN,32,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,дополнительное образование
7,0,NaN,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,NaN,образование
...,...,...,...,...,...,...,...,...,...,...,...,...
21515,1,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,заняться образованием
21519,1,NaN,37,ученая степень,4,в разводе,3,M,сотрудник,0,NaN,покупка коммерческой недвижимости
21522,1,NaN,38,среднее,1,гражданский брак,1,M,сотрудник,1,NaN,недвижимость
21523,3,NaN,38,среднее,1,женат / замужем,0,M,сотрудник,1,NaN,на покупку своего автомобиля


children                0
days_employed       11119
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income        11119
purpose                 0
dtype: int64

nan

nan

In [9]:
# разбиваем пропуски по типу занятости
companion_income_type=data[data['income_type'] == 'компаньон']
display (companion_income_type)
display(data[data['income_type'] == 'компаньон'].isna().sum())
companion_total_income_mean=data[data['income_type'] == 'компаньон']['total_income'].mean()
companion_days_employed_mean=data[data['income_type'] == 'компаньон']['days_employed'].mean()
display(companion_total_income_mean)
display(companion_days_employed_mean)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5,0,NaN,27,высшее,0,гражданский брак,1,M,компаньон,0,NaN,покупка жилья
6,0,NaN,43,высшее,0,женат / замужем,0,F,компаньон,0,NaN,операции с жильем
10,2,NaN,36,высшее,0,женат / замужем,0,M,компаньон,0,NaN,покупка недвижимости
14,0,NaN,56,высшее,0,гражданский брак,1,F,компаньон,1,NaN,покупка жилой недвижимости
33,0,NaN,48,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21512,0,NaN,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,NaN,получение дополнительного образования
21514,0,NaN,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,NaN,строительство недвижимости
21516,0,NaN,42,высшее,0,женат / замужем,0,F,компаньон,0,NaN,покупка своего жилья
21517,0,NaN,42,высшее,0,гражданский брак,1,F,компаньон,0,NaN,на покупку своего автомобиля


children               0
days_employed       5085
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        5085
purpose                0
dtype: int64

nan

nan

In [10]:
# разбиваем пропуски по типу занятости
civil_servant_income_type=data[data['income_type'] == 'госслужащий']
display (civil_servant_income_type)
display(data[data['income_type'] == 'госслужащий'].isna().sum())
civil_servant_total_income_mean=data[data['income_type'] == 'госслужащий']['total_income'].mean()
civil_servant_days_employed_mean=data[data['income_type'] == 'госслужащий']['days_employed'].mean()
display(civil_servant_total_income_mean)
display(civil_servant_days_employed_mean)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
47,1,NaN,33,высшее,0,гражданский брак,1,M,госслужащий,0,NaN,на проведение свадьбы
62,0,NaN,48,Высшее,0,Не женат / не замужем,4,M,госслужащий,0,NaN,получение высшего образования
70,1,NaN,28,ВЫСШЕЕ,0,женат / замужем,0,M,госслужащий,0,NaN,покупка коммерческой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21288,0,NaN,34,Среднее,1,Не женат / не замужем,4,M,госслужащий,0,NaN,жилье
21349,0,NaN,52,среднее,1,вдовец / вдова,2,F,госслужащий,0,NaN,покупка жилья
21380,0,NaN,54,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,получение высшего образования
21459,0,NaN,48,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,на покупку своего автомобиля


children               0
days_employed       1459
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1459
purpose                0
dtype: int64

nan

nan

In [11]:
# разбиваем пропуски по типу занятости
civil_servant_income_type=data[data['income_type'] == 'безработный']
display (civil_servant_income_type)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,NaN,31,среднее,1,женат / замужем,0,M,безработный,1,NaN,покупка жилья для сдачи
14798,0,NaN,45,Высшее,0,гражданский брак,1,F,безработный,0,NaN,ремонт жилью


In [12]:
# разбиваем пропуски по типу занятости
civil_servant_income_type=data[data['income_type'] == 'предприниматель']
display (civil_servant_income_type)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,NaN,27,высшее,0,гражданский брак,1,F,предприниматель,0,NaN,на проведение свадьбы


In [13]:
# разбиваем пропуски по типу занятости
civil_servant_income_type=data[data['income_type'] == 'в декрете']
display (civil_servant_income_type)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20845,2,NaN,39,СРЕДНЕЕ,1,женат / замужем,0,F,в декрете,1,NaN,автомобиль


In [14]:
# разбиваем пропуски по типу занятости
civil_servant_income_type=data[data['income_type'] == 'студент']
display (civil_servant_income_type)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9410,0,NaN,22,высшее,0,Не женат / не замужем,4,M,студент,0,NaN,строительство собственной недвижимости


Все остальные пропуски забиваем нулями.
В качестве пропусков в нашем датасете выступают **общий трудовой стаж** и **ежемесячный доход**
заполним пропуски в **total_income** и **days_employed** нулями



In [15]:
data['total_income'] = data['total_income'].fillna(0)
data['days_employed'] = data['days_employed'].fillna(0)
display(data.head(5))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0.000000,42,высшее,0,женат / замужем,0,F,сотрудник,0,0.00000,покупка жилья
1,1,0.000000,36,среднее,1,женат / замужем,0,F,сотрудник,0,0.00000,приобретение автомобиля
2,0,0.000000,33,Среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,покупка жилья
3,3,0.000000,32,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.07787,сыграть свадьбу


In [16]:
display(data['family_status'].unique(),data['family_status_id'].unique())# проверка семейного положения
display(data['family_status'].value_counts(),data['family_status_id'].value_counts())# подсчет количества по семейного положения

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array([0, 1, 2, 3, 4])

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [17]:
display(data['children'].unique())# проверка количества детей
display(data['children'].value_counts())# подсчет количества детей

array([ 1,  0,  3,  2, -1,  4, 20,  5])

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [18]:
display (data.loc[data['children']>=20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,0.0,21,среднее,1,женат / замужем,0,M,компаньон,0,0.0,покупка жилья
720,20,0.0,44,среднее,1,женат / замужем,0,F,компаньон,0,0.0,покупка недвижимости
1074,20,0.0,56,среднее,1,женат / замужем,0,F,сотрудник,1,0.0,получение образования
2510,20,0.0,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,0.0,операции с коммерческой недвижимостью
2941,20,0.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,0.0,40,среднее,1,женат / замужем,0,F,сотрудник,1,0.0,свой автомобиль
21325,20,0.0,37,среднее,1,женат / замужем,0,F,компаньон,0,0.0,профильное образование
21390,20,0.0,53,среднее,1,женат / замужем,0,M,компаньон,0,0.0,покупка жилой недвижимости
21404,20,0.0,52,среднее,1,женат / замужем,0,M,компаньон,0,0.0,операции со своей недвижимостью


Проверка уникальных значений по типу занятости, семейному положению, детям выявили, что количестов детей может быть как -1 так и 20 шт. Вероятнее всего это опечатки т.к -1 ребенок это оригинально, а 20 шт может быть если это приемные. Заменим -1 на 1, а 20 на 2.

In [19]:
for i in range(len(data)):
    if data.loc[i,'children']<0:
        data.loc[i,'children']=1
    if data.loc[i,'children']==20:
        data.loc[i,'children']=2
        
display(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Анализ столбца **days_employed** показал,что есть отрицательные значения - это является опечаткой и большие положительные значения. Анализируя большие положительные значения можно сказать, что скорее всего данные представлены не в рабочи днях, а в часах. Поэтому большие положительные значения переведем в дни, а отрицательные в положительные. 

In [20]:
for i in range(len(data)):
    if data.loc[i,'days_employed']>0:
        data.loc[i,'days_employed']=data.loc[i,'days_employed']/24
    if data.loc[i,'days_employed']<0:
        data.loc[i,'days_employed']=abs (data.loc[i,'days_employed'])



In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Проведенные замены позволили заменить все пропуски, отрицательные значения и выбросы из выборки. Это позволит проводить дальнейшие операции с данными

### Замена типа данных

In [22]:
data['days_employed'] = data['days_employed'].astype('int')# перевод из вещественных чисел в целые для рабочих дней
display(data.head(10))# вывод первых 10 значений

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,42,высшее,0,женат / замужем,0,F,сотрудник,0,0.00000,покупка жилья
1,1,0,36,среднее,1,женат / замужем,0,F,сотрудник,0,0.00000,приобретение автомобиля
2,0,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,покупка жилья
3,3,0,32,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.07787,сыграть свадьбу
5,0,0,27,высшее,0,гражданский брак,1,M,компаньон,0,0.00000,покупка жилья
6,0,0,43,высшее,0,женат / замужем,0,F,компаньон,0,0.00000,операции с жильем
7,0,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,0.00000,образование
8,2,0,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,0.00000,на проведение свадьбы
9,0,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,покупка жилья для семьи


**Вывод**

Заменили вещественный тип данных на целочисленный с помощью метода `astype('int')`

### Обработка дубликатов

In [23]:
data['education'] = data['education'].str.lower()
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,42,высшее,0,женат / замужем,0,F,сотрудник,0,0.00000,покупка жилья
1,1,0,36,среднее,1,женат / замужем,0,F,сотрудник,0,0.00000,приобретение автомобиля
2,0,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,покупка жилья
3,3,0,32,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.07787,сыграть свадьбу


In [24]:
display (data.duplicated().sum())#сумма дубликатов


2629

In [25]:
duplicated_data = data[data.duplicated()].head(10) #результат — датафрейм с дубликатами
display(duplicated_data) 
data=data.drop_duplicates()
data = data.dropna().reset_index(drop=True) 
display (data.duplicated().sum())#сумма дубликатов

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
427,0,0,49,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,строительство недвижимости
473,0,0,45,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,на покупку подержанного автомобиля
858,0,0,41,среднее,1,женат / замужем,0,F,госслужащий,0,0.0,покупка своего жилья
860,0,0,27,среднее,1,Не женат / не замужем,4,F,сотрудник,0,0.0,на покупку автомобиля
904,1,0,47,среднее,1,гражданский брак,1,M,сотрудник,0,0.0,свадьба
1016,0,0,52,высшее,0,женат / замужем,0,M,компаньон,0,0.0,покупка недвижимости
1122,0,0,48,среднее,1,Не женат / не замужем,4,F,сотрудник,0,0.0,покупка коммерческой недвижимости
1216,0,0,49,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,строительство собственной недвижимости
1226,0,0,42,среднее,1,женат / замужем,0,M,сотрудник,0,0.0,сделка с автомобилем
1247,0,0,54,среднее,1,гражданский брак,1,M,сотрудник,0,0.0,свадьба


0

**Вывод**

проверка дубликатов показала, что количество дубликатов составляет 2629 шт. У этих заемщиков нет данных по ежемесяному доход, нет задолжности, нет рабочих дней. В принципе для кредитного отдела они не интересны и их можно удалить

### Лемматизация

In [26]:
#Определяем частотность слов
purpose_text=data['purpose']# столбец по цели
display(purpose_text)
purpose_text_sum=(purpose_text+' ').sum()# объеденяем все в одно целое и добавляем пробелы

lemmas_n=m.lemmatize(purpose_text_sum)# лемматизируем

display(Counter(lemmas_n))# считаем частоту слов
# может можно как-то по другому

0                       покупка жилья
1             приобретение автомобиля
2                       покупка жилья
3          дополнительное образование
4                     сыграть свадьбу
                     ...             
18891               операции с жильем
18892            сделка с автомобилем
18893                    недвижимость
18894    на покупку своего автомобиля
18895           на покупку автомобиля
Name: purpose, Length: 18896, dtype: object

Counter({'покупка': 5192,
         ' ': 48676,
         'жилье': 3907,
         'приобретение': 428,
         'автомобиль': 3912,
         'дополнительный': 857,
         'образование': 3692,
         'сыграть': 589,
         'свадьба': 1762,
         'операция': 2331,
         'с': 2633,
         'на': 1896,
         'проведение': 586,
         'для': 1135,
         'семья': 564,
         'недвижимость': 5623,
         'коммерческий': 1182,
         'жилой': 1076,
         'строительство': 1643,
         'собственный': 565,
         'подержать': 785,
         'свой': 1975,
         'со': 565,
         'заниматься': 834,
         'сделка': 867,
         'получение': 1221,
         'высокий': 1252,
         'подержанный': 100,
         'профильный': 390,
         'сдача': 571,
         'ремонт': 544,
         ' \n': 1})

In [27]:
data_lemmas=[]# список для лемматизированных данных
purpose_id=[]# список purpose_id для наглядности
purpose_lemmas=[]# расшифровка лемматизированных данных
for i in range(len(data['purpose'])):
    lemmas=m.lemmatize(data['purpose'][i])
    data_lemmas.append(lemmas)
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        purpose_id.append(1)
        purpose_lemmas.append('жилье')
    elif 'образование' in lemmas:
        purpose_id.append(2)
        purpose_lemmas.append('образование')
    elif 'автомобиль' in lemmas:
        purpose_id.append(3)
        purpose_lemmas.append('автомобиль')
    elif 'свадьба' in lemmas:
        purpose_id.append(4)
        purpose_lemmas.append('свадьба')
    else:
        purpose_id.append(5)
        

        
data['purpose_id']=purpose_id
data['purpose_lemmas']=purpose_lemmas
display(data.head(10))


display(data['purpose_id'].value_counts())# подсчет количества purpose_id 
display('Всего заявок на кредит:',data['purpose_id'].value_counts().sum())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,purpose_lemmas
0,1,0,42,высшее,0,женат / замужем,0,F,сотрудник,0,0.00000,покупка жилья,1,жилье
1,1,0,36,среднее,1,женат / замужем,0,F,сотрудник,0,0.00000,приобретение автомобиля,3,автомобиль
2,0,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,покупка жилья,1,жилье
3,3,0,32,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,дополнительное образование,2,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.07787,сыграть свадьбу,4,свадьба
5,0,0,27,высшее,0,гражданский брак,1,M,компаньон,0,0.00000,покупка жилья,1,жилье
6,0,0,43,высшее,0,женат / замужем,0,F,компаньон,0,0.00000,операции с жильем,1,жилье
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,образование,2,образование
8,2,0,35,высшее,0,гражданский брак,1,F,сотрудник,0,0.00000,на проведение свадьбы,4,свадьба
9,0,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,0.00000,покупка жилья для семьи,1,жилье


1    9530
3    3912
2    3692
4    1762
Name: purpose_id, dtype: int64

'Всего заявок на кредит:'

18896

**Вывод**

Лемматизировали цели кредита, определили что всего 4 цели (жилье, образование, автомобиль, свадьба). При лемматизации дополнительно были выявлены слова (операция - относится к недвижимости; строительство, ремонт тоже отностися к недвижимости) поэтому всего 4 цели кредита

### Категоризация данных

In [28]:
# сгруппируем таблицу по признаку задолженности и от количества детей
data_children_grouped = data.groupby(['children', 'debt']).agg({'debt':'count'})

# рассчитаем % по типу наличия задолженности от общего количества кредитов
data_children_grouped['percentage_%']=(data_children_grouped['debt']/(data_children_grouped.groupby('children')['debt'].transform('sum')))*100
display(data_children_grouped)

debt  percentage_%
children debt                     
0        0     11229     91.530812
         1      1039      8.469188
1        0      3907     90.002304
         1       434      9.997696
2        0      1716     89.655172
         1       198     10.344828
3        0       297     91.666667
         1        27      8.333333
4        0        36     90.000000
         1         4     10.000000
5        0         9    100.000000

In [29]:
# сгруппируем таблицу по признаку задолженности от семейного положения
data_family_status_grouped = data.groupby(['family_status_id', 'debt']).agg({'debt':'count'})
data_family_status_grouped['percentage_%']=(data_family_status_grouped['debt']/(data_family_status_grouped.groupby('family_status_id')['debt'].transform('sum')))*100
display(data_family_status_grouped)

debt  percentage_%
family_status_id debt                    
0                0     9659     91.398562
                 1      909      8.601438
1                0     3167     89.337094
                 1      378     10.662906
2                0      884     93.347413
                 1       63      6.652587
3                0     1084     92.728828
                 1       85      7.271172
4                0     2400     89.988751
                 1      267     10.011249

In [30]:
# сгруппируем таблицу по признаку задолженности от уровня дохода
data_total_income_grouped = data.groupby(['total_income', 'debt']).agg({'debt':'count'})
data_total_income_grouped['percentage_%']=(data_total_income_grouped['debt']/(data_total_income_grouped.groupby('total_income')['debt'].transform('sum')))*100
display(data_total_income_grouped)

debt  percentage_%
total_income  debt                     
0.000000      0     13581     90.137386
              1      1486      9.862614
20667.263793  1         1    100.000000
21205.280566  0         1    100.000000
21695.101789  0         1    100.000000
...                   ...           ...
644077.836518 0         1    100.000000
681300.590458 0         1    100.000000
694150.890876 0         1    100.000000
708927.197617 0         1    100.000000
735103.270167 0         1    100.000000

[3446 rows x 2 columns]

In [31]:
# сгруппируем таблицу по признаку задолженности от типа занятости
data_income_type_grouped = data.groupby(['income_type', 'debt']).agg({'debt':'count'})
data_income_type_grouped['percentage_%']=(data_income_type_grouped['debt']/(data_income_type_grouped.groupby('income_type')['debt'].transform('sum')))*100
display(data_income_type_grouped)

debt  percentage_%
income_type     debt                    
безработный     0        1     50.000000
                1        1     50.000000
в декрете       1        1    100.000000
госслужащий     0     1317     93.870278
                1       86      6.129722
компаньон       0     4250     91.931646
                1      373      8.068354
пенсионер       0     3613     94.358840
                1      216      5.641160
предприниматель 0        2    100.000000
сотрудник       0     8010     88.655230
                1     1025     11.344770
студент         0        1    100.000000

In [32]:
# сгруппируем таблицу по признаку задолженности от типа цели
data_purpose_id_grouped = data.groupby(['purpose_id','purpose_lemmas', 'debt']).agg({'debt':'count'})
data_purpose_id_grouped['percentage_%']=(data_purpose_id_grouped['debt']/(data_purpose_id_grouped.groupby('purpose_id')['debt'].transform('sum')))*100
display(data_purpose_id_grouped)

debt  percentage_%
purpose_id purpose_lemmas debt                    
1          жилье          0     8768     92.004197
                          1      762      7.995803
2          образование    0     3326     90.086674
                          1      366      9.913326
3          автомобиль     0     3515     89.851738
                          1      397     10.148262
4          свадьба        0     1585     89.954597
                          1      177     10.045403

**Вывод**

Провели категоризацию данных. Получили соотношение отданных кредитов от просроченных

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

В принципе зависимость между наличием детей и возвратом кредита в срок есть но не знаительная, все клиенты стараются вернуть кредит, что характерно для 90% клиентов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

Данные для оценки зависимости между уровнем дохода и возвратом кредита в срок не показательны, так как могут вернуть как с малым количеством денег, так и не вернуть с большим количеством денег

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

Данные для оценки зависимости между уровнем дохода и возвратом кредита в срок не показательны, так как могут вернуть как с малым количеством денег, так и не вернуть с большим количеством денег

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

Цели кредита в принципе не влияют на возврат все в рамках 90%

## Шаг 4. Общий вывод

Провели анализ платежеспособности клиентов, по данным получены следующие показатели:
Возвращают кредиты (предприниматель - 100%; пенсионер - 94,4%; госслужащий - 94%; компаньон - 92%; сотрудник -88%).
Не возвращают кредиты **студенты**, **безработные**, **в декрете**

Проверили четыре гипотезы и установили:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок? 

Первая гипотеза подтвердилась лишь частично.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вторая гипотеза подтвердилась лишь отчасти. 

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Третья гипотеза не подтвердилась на основной массе пользователей они незаметны.

4. Как разные цели кредита влияют на его возврат в срок?

Четвертая гипотеза подтвердилась лишь отчасти. 